In [1]:
import bempp.api
import numpy as np
import dolfin

In [2]:
from dolfin import *     
from dolfin_utils.meshconvert import meshconvert

theta = 10.0
pow_bnd = 1.0

FEM_u_error = []
BEM_u_error = []
BEM_lambda_error = []
error_conv = []
error_between = []

mesh_size = []
iter_num = []
iter_num_no_prec = []

dir_imp = False
trace_variable = True

########################################################################
grid_size = [6, 5, 4, 3, 2]#, 1]
 
tol=1e-5

    
# ################################### Example 1 #############################################    
def f(x, n, d, res):
    res[0] = 1/(4.0*np.pi*np.sqrt(x[0]**2+x[1]**2+x[2]**2)) 
    
def g(x, n, d, res):
    grad = np.array([
            -x[0]/(4.0*np.pi*(x[0]**2+x[1]**2+x[2]**2)**(3.0/2.0)),
            -x[1]/(4.0*np.pi*(x[0]**2+x[1]**2+x[2]**2)**(3.0/2.0)),
            -x[2]/(4.0*np.pi*(x[0]**2+x[1]**2+x[2]**2)**(3.0/2.0))
        ])
    res[0] = np.dot(grad, n)
    
# ################################### Example 2 #############################################    
# def f(x, n, d, res):
#     res[0] = 1/(np.sqrt(x[0]**2+x[1]**2+x[2]**2)) 

# def g(x, n, d, res):
#     grad = np.array([
#             -x[0]/((x[0]**2+x[1]**2+x[2]**2)**(3.0/2.0)),
#             -x[1]/((x[0]**2+x[1]**2+x[2]**2)**(3.0/2.0)),
#             -x[2]/((x[0]**2+x[1]**2+x[2]**2)**(3.0/2.0))
#         ])
#     res[0] = np.dot(grad, n)


In [ ]:
from dolfin import *          
from dolfin_utils.meshconvert import meshconvert
import pylab as plt
from bempp.api import fenics_interface
from bempp.api.fenics_interface import FenicsOperator
from scipy.sparse.linalg.interface import LinearOperator
from scipy.sparse.linalg import gmres

parameters["allow_extrapolation"] = True 

for m in grid_size:
    ######################### Mesh and spaces ##################################
    mesh_file = "Sphere_h_0_"+format(m)+".msh"
    meshconvert.convert2xml(mesh_file, "mesh.xml", iformat=None)
    mesh = dolfin.Mesh("mesh.xml")
    
    fenics_space = dolfin.FunctionSpace(mesh, "Lagrange", 1)
    bnd = dolfin.BoundaryMesh(mesh, 'exterior')
    trace_space, trace_matrix  = fenics_interface.coupling.fenics_to_bempp_trace_data(fenics_space)
    bempp_space = bempp.api.function_space(trace_space.grid, "P", 1)
    
    ######################### Matrices #########################
    dir_identity_on_bem = bempp.api.operators.boundary.sparse.identity(trace_space, trace_space, bempp_space)
    dir_identity_on_trace = bempp.api.operators.boundary.sparse.identity(trace_space, trace_space, trace_space)

    neu_identity_on_trace = bempp.api.operators.boundary.sparse.identity(bempp_space, bempp_space, trace_space)
    neu_identity_on_bem = bempp.api.operators.boundary.sparse.identity(bempp_space, bempp_space, bempp_space)

    mass = bempp.api.operators.boundary.sparse.identity(trace_space, bempp_space, trace_space)
    op_mass = bempp.api.operators.boundary.sparse.identity(bempp_space, trace_space, trace_space)

    dlp = bempp.api.operators.boundary.laplace.double_layer(trace_space, trace_space, bempp_space)
    slp = bempp.api.operators.boundary.laplace.single_layer(bempp_space, trace_space, bempp_space)
    adlp = bempp.api.operators.boundary.laplace.adjoint_double_layer(bempp_space, bempp_space, trace_space)
    hslp = bempp.api.operators.boundary.laplace.hypersingular(trace_space, bempp_space, trace_space)

    # Preconditioner
    mass_P = bempp.api.operators.boundary.sparse.identity(trace_space, bempp_space, bempp_space)
    op_mass_P = bempp.api.operators.boundary.sparse.identity(bempp_space, trace_space, bempp_space)

    slp_P  = bempp.api.operators.boundary.laplace.single_layer(bempp_space, trace_space, trace_space)
    hslp_P = bempp.api.operators.boundary.laplace.hypersingular(trace_space, bempp_space, bempp_space)
    
    
    ######################### FEM variables ##################################
    u = dolfin.TrialFunction(fenics_space)
    v = dolfin.TestFunction(fenics_space)

    nor = dolfin.FacetNormal(mesh)
    h = dolfin.CellDiameter(bnd)

#     h1 = []
#     for cell in cells(bnd):
#         h1.append(cell.inradius())

#     hmax = max(h1)

    hmax = 0
    for element in  list(trace_space.grid.leaf_view.entity_iterator(0)):
        hmax = max(hmax,element.geometry.volume**(0.5))

    print(hmax)
    mesh_size.append(hmax)

    ################################### Example 1 #############################################
#     hybrid         -->  FINE
#     hybrid + weak  -->  H1 BEM problem
#     Nitsche        -->  
#     Nitsche + weak -->  
    u_D = dolfin.Function(fenics_space)
    u_Dir = dolfin.Expression('-(x[0]*x[0]+x[1]*x[1]+x[2]*x[2] - 3)/(8*pi)', degree=2) 
    u_D.interpolate(u_Dir)
    
    rhs_f = Constant(3./(4.*pi))

#     ################################### Example 2 #############################################
# #     hybrid         -->  FINE
# #     hybrid + weak  -->  
# #     Nitsche        -->  
# #     Nitsche + weak -->  
#     u_D = dolfin.Function(fenics_space)
#     u_Dir = dolfin.Expression('sin(pi*(x[0]*x[0]+x[1]*x[1]+x[2]*x[2]))/(2*pi) + cos(pi*(x[0]*x[0]+x[1]*x[1]+x[2]*x[2]))/(2*pi) + (2*pi+1)/(2*pi)', degree=2) 
#     u_D.interpolate(u_Dir)
    
#     rhs_f = dolfin.Expression('sin(pi*(x[0]*x[0]+x[1]*x[1]+x[2]*x[2]))*(2*pi*(x[0]*x[0]+x[1]*x[1]+x[2]*x[2])+3) + cos(pi*(x[0]*x[0]+x[1]*x[1]+x[2]*x[2]))*(2*pi*(x[0]*x[0]+x[1]*x[1]+x[2]*x[2])-3)', degree=2) 
    
    
    ######################### BEM variables ##################################
    f_fun = bempp.api.GridFunction(trace_space, fun=f)
    g_fun = bempp.api.GridFunction(bempp_space, fun=g)
    
    beta = theta/hmax
    
    ######################### RHS ##################################
    FEM_rhs = v*rhs_f*dolfin.dx()  + v*u_D*dolfin.dx()  #+ beta*v*u_D*dolfin.ds() #- dolfin.dot(dolfin.grad(v),nor)*u_D*dolfin.ds 

    # The rhs from the FEM
    rhs_fem = dolfin.assemble(FEM_rhs)
    # rhs_fem = np.zeros(mesh.num_vertices())

    # The rhs from the BEM
    if trace_variable:
        rhs_bem = np.zeros(2*trace_space.global_dof_count+bempp_space.global_dof_count)
        if dir_imp:
            rhs_bem += np.concatenate([np.zeros(trace_space.global_dof_count+bempp_space.global_dof_count), beta**pow_bnd*f_fun.projections(trace_space)])
    else:
        rhs_bem = np.concatenate([np.zeros(bempp_space.global_dof_count), beta*f_fun.projections(trace_space)]) #np.zeros(trace_space.global_dof_count+bempp_space.global_dof_count)
        if dir_imp:
            rhs_bem += np.concatenate([np.zeros(bempp_space.global_dof_count), beta**pow_bnd*f_fun.projections(trace_space)])


    # The combined rhs
    rhs = np.concatenate([rhs_fem, rhs_bem])
    
    ######################### LHS ##################################
    trace_op = LinearOperator(trace_matrix.shape, lambda x:trace_matrix*x)
    trace_T_op = LinearOperator(trace_matrix.T.shape, lambda x:trace_matrix.T*x)

    A = FenicsOperator((dolfin.dot(dolfin.grad(u),dolfin.grad(v))) * dolfin.dx() \
                        + u*v*dolfin.dx() \
                        - dolfin.dot(dolfin.grad(u),nor)*v*dolfin.ds \
                        - dolfin.dot(dolfin.grad(v),nor)*u*dolfin.ds \
                        + beta*v*u*dolfin.ds() \
                      )
    
    B =  FenicsOperator(dolfin.dot(dolfin.grad(u),nor)*v*dolfin.ds() 
                        - beta*v*u*dolfin.ds())
    BT = FenicsOperator(dolfin.dot(dolfin.grad(v),nor)*u*dolfin.ds() 
                        - beta*v*u*dolfin.ds())
    C =  FenicsOperator(beta*v*u*dolfin.ds() )#+ beta**4*v*u*dolfin.ds)

    # The rhs from the BEM
    if trace_variable:
        blocks = [[None,None,None,None],[None,None,None,None],[None,None,None,None],[None,None,None,None]]

        blocks[0][0] =  A.weak_form()
        blocks[0][3] =  BT.weak_form()*trace_T_op
        blocks[3][0] =  trace_op*B.weak_form()

        blocks[1][1] =  slp.weak_form()
        blocks[1][2] =  (-0.5 * dir_identity_on_bem - dlp).weak_form()
        blocks[1][3] =  dir_identity_on_bem.weak_form()

        blocks[2][1] =  (0.5*neu_identity_on_trace + adlp).weak_form()
        blocks[2][2] =  (beta*mass + hslp).weak_form()     
        blocks[2][3] =  -beta*mass.weak_form()

        blocks[3][1] =  -op_mass.weak_form()
        blocks[3][2] =  -beta*dir_identity_on_trace.weak_form()
        blocks[3][3] =  trace_op*C.weak_form()*trace_T_op + beta*dir_identity_on_trace.weak_form() 

        if dir_imp:
            blocks[3][3] += beta**pow_bnd*dir_identity_on_trace.weak_form()
    else:
        blocks = [[None,None,None],[None,None,None],[None,None,None]]
        blocks[0][0] =  A.weak_form()
        blocks[1][1] =  slp.weak_form()
        blocks[1][2] =  (-0.5 * dir_identity_on_bem - dlp).weak_form()
        blocks[2][1] =  (0.5*neu_identity_on_trace + adlp).weak_form()
        blocks[2][2] =  (beta*mass + hslp).weak_form()

        if dir_imp:
            blocks[2][2] += beta**pow_bnd*mass.weak_form()

    blocked = bempp.api.BlockedDiscreteOperator(np.array(blocks))
    
    ######################### Preconditioner ##################################
    
    if trace_variable:
        # Compute the sparse inverse of the Helmholtz operator
        # Although it is not a boundary operator we can use
        # the SparseInverseDiscreteBoundaryOperator function from
        # BEM++ to turn its LU decomposition into a linear operator.
#         P1 = bempp.api.InverseSparseDiscreteBoundaryOperator(
#             blocked[0,0].sparse_operator.tocsc())
        

        # For BEM part
        P11 = (beta * mass_P+hslp_P).weak_form()
        P12 = (dlp+0.5 * dir_identity_on_bem).weak_form()
        P21 = (-adlp-0.5 * neu_identity_on_trace).weak_form()
        P22 = slp_P.weak_form()    

        # Inverse of the mass 
        invmassD = bempp.api.InverseSparseDiscreteBoundaryOperator(dir_identity_on_trace.weak_form())
        invmassN = bempp.api.InverseSparseDiscreteBoundaryOperator(neu_identity_on_bem.weak_form())     
        
        invtrace = bempp.api.InverseSparseDiscreteBoundaryOperator(beta*dir_identity_on_trace.weak_form() )

        TinvmassD = invmassD.T
        TinvmassN = invmassN.T  

        # Create a block diagonal preconditioner object using the Scipy LinearOperator class
        def apply_prec(x):
            """Apply the block diagonal preconditioner"""
            m1 = A.weak_form().shape[0]
            m2 = P11.shape[0]
            m3 = P22.shape[0]
            m4 = invtrace.shape[0]
            n1 = A.weak_form().shape[1]
            n2 = P11.shape[1]
            n3 = P22.shape[1]
            n4 = invtrace.shape[1]

            res1 = x[:n1] #P1.dot(x[:n1])
            res2 = TinvmassD.dot(P11.dot(invmassD.dot(x[n1:n1+n2])) + P12.dot(invmassN.dot(x[n1+n2:n1+n2+n3])))  
            res3 = TinvmassN.dot(P22.dot(invmassN.dot(x[n1+n2:n1+n2+n3])) + P21.dot(invmassD.dot(x[n1:n1+n2]))) 
            res4 = invtrace.dot(x[n1+n2+n3:]) 
            return np.concatenate([res1, res2, res3, res4])

        p_shape = (A.weak_form().shape[0]+P11.shape[0]+P22.shape[0]+invtrace.shape[0], A.weak_form().shape[1]+P11.shape[1]+P22.shape[1]+invtrace.shape[1])
        P = LinearOperator(p_shape, apply_prec, dtype=np.dtype('complex128'))
        
        def apply_mass(x):
            """Apply the block diagonal preconditioner"""
            m1 = A.weak_form().shape[0]
            m2 = invmassD.shape[0]
            m3 = invmassN.shape[0]
            m4 = invtrace.shape[0]
            n1 = A.weak_form().shape[1]
            n2 = invmassD.shape[1]
            n3 = invmassN.shape[1]
            n4 = invtrace.shape[1]

            res1 = x[:n1] #P1.dot(x[:n1])
            res2 = invmassD.dot(x[n1:n1+n2])
            res3 = invmassN.dot(x[n1+n2:n1+n2+n3])
            res4 = invtrace.dot(x[n1+n2+n3:]) 
            return np.concatenate([res1, res2, res3, res4])

        mass_shape = (A.weak_form().shape[0]+invmassD.shape[0]+invmassN.shape[0]+invtrace.shape[0], A.weak_form().shape[1]+invmassD.shape[1]+invmassN.shape[1]+invtrace.shape[1])
        Mass = LinearOperator(mass_shape, apply_mass, dtype=np.dtype('complex128'))
    else:
        # Compute the sparse inverse of the Helmholtz operator
        # Although it is not a boundary operator we can use
        # the SparseInverseDiscreteBoundaryOperator function from
        # BEM++ to turn its LU decomposition into a linear operator.
#         P1 = bempp.api.InverseSparseDiscreteBoundaryOperator(
#             blocked[0,0].sparse_operator.tocsc())

        # For BEM part
        P11 = (beta * mass_P+hslp_P).weak_form()
        P12 = (dlp+0.5 * dir_identity_on_bem).weak_form()
        P21 = (-adlp-0.5 * neu_identity_on_trace).weak_form()
        P22 = slp_P.weak_form()    

        # Inverse of the mass 
        invmassD = bempp.api.InverseSparseDiscreteBoundaryOperator(dir_identity_on_trace.weak_form())
        invmassN = bempp.api.InverseSparseDiscreteBoundaryOperator(neu_identity_on_bem.weak_form())      

        TinvmassD = invmassD.T
        TinvmassN = invmassN.T  

        # Create a block diagonal preconditioner object using the Scipy LinearOperator class
        def apply_prec(x):
            """Apply the block diagonal preconditioner"""            
            m1 = A.weak_form().shape[0]
            m2 = P11.shape[0]
            m3 = P22.shape[0]
            n1 = A.weak_form().shape[1]
            n2 = P11.shape[1]
            n3 = P22.shape[1]

            res1 = x[:n1] #P1.dot(x[:n1])
            res2 = P11.dot(x[n1:n1+n2]) + P12.dot(x[n1+n2:n1+n2+n3]) #P11.dot(x[n1:n1+n2]) + P12.dot(op_mass.weak_form().dot(x[n1+n2:n1+n2+n3])) 
            res3 = P22.dot(x[n1+n2:n1+n2+n3]) + P21.dot(x[n1:n1+n2]) #P22.dot(x[n1+n2:n1+n2+n3]) + P21.dot(mass_P.weak_form().dot(x[n1:n1+n2]))  
            return np.concatenate([res1, res2, res3])

        p_shape = (A.weak_form().shape[0]+P11.shape[0]+P22.shape[0], A.weak_form().shape[1]+P11.shape[1]+P22.shape[1])
        P = LinearOperator(p_shape, apply_prec, dtype=np.dtype('complex128'))
        
        def apply_mass(x):
            """Apply the block diagonal preconditioner"""
            m1 = A.weak_form().shape[0]
            m2 = invmassD.shape[0]
            m3 = invmassN.shape[0]
            n1 = A.weak_form().shape[1]
            n2 = invmassD.shape[1]
            n3 = invmassN.shape[1]

            res1 = x[:n1] #P1.dot(x[:n1])
            res2 = invmassD.dot(x[n1:n1+n2])
            res3 = invmassN.dot(x[n1+n2:n1+n2+n3])
            return np.concatenate([res1, res2, res3])

        mass_shape = (A.weak_form().shape[0]+invmassD.shape[0]+invmassN.shape[0], A.weak_form().shape[1]+invmassD.shape[1]+invmassN.shape[1])
        Mass = LinearOperator(mass_shape, apply_mass, dtype=np.dtype('complex128'))
        
    ######################## Solver ##################################
    # Create a callback function to count the number of iterations
    it_count_prec = 0
    def count_iterations_prec(x):
        global it_count_prec
        it_count_prec += 1
        
    soln_prec, info = gmres(blocked, rhs, M = P, tol = tol, callback=count_iterations_prec)
    
    soln_prec_fem = soln_prec[:fenics_space.dim()]
    soln_prec_lambda_bem = soln_prec[fenics_space.dim():bempp_space.global_dof_count+fenics_space.dim()]
    soln_prec_u_bem = soln_prec[bempp_space.global_dof_count+fenics_space.dim():bempp_space.global_dof_count+trace_space.global_dof_count+fenics_space.dim()]
    
    iter_num.append(it_count_prec)
    print("Number of iterations with preconditioner: {0}".format(it_count_prec))
    
    # Create a callback function to count the number of iterations
    it_count = 0
    def count_iterations(x):
        global it_count
        it_count += 1
        
    soln, info = gmres(blocked, rhs, M = Mass, tol = tol, callback=count_iterations)

    soln_fem = soln[:fenics_space.dim()]
    soln_lambda_bem = soln[fenics_space.dim():bempp_space.global_dof_count+fenics_space.dim()]
    soln_u_bem = soln[bempp_space.global_dof_count+fenics_space.dim():bempp_space.global_dof_count+trace_space.global_dof_count+fenics_space.dim()]
    
    iter_num_no_prec.append(it_count)
    print("Number of iterations without preconditioner: {0}".format(it_count))

    ######################### Error ##################################
    # Store the real part of the FEM solution
    u = dolfin.Function(fenics_space)
    u.vector()[:] = np.ascontiguousarray(np.real(soln_fem))
    u.vector()[:] = np.ascontiguousarray(np.real(soln_prec_fem))

    # Solution function with dirichlet data on the boundary
    dirichlet_data = trace_matrix * soln_fem
    dirichlet_fun_fem = bempp.api.GridFunction(trace_space, coefficients=dirichlet_data)
    dirichlet_fun = bempp.api.GridFunction(trace_space, coefficients=soln_u_bem)
    
    dirichlet_data = trace_matrix * soln_prec_fem
    dirichlet_fun_fem = bempp.api.GridFunction(trace_space, coefficients=dirichlet_data)
    dirichlet_fun = bempp.api.GridFunction(trace_space, coefficients=soln_prec_u_bem)

    # Solution function with Neumann data on the boundary
    neumann_fun = bempp.api.GridFunction(bempp_space, coefficients=soln_lambda_bem)
    neumann_fun = bempp.api.GridFunction(bempp_space, coefficients=soln_prec_lambda_bem)

    error = (u_D-u)**2*dolfin.dx()
    L2_error = dolfin.assemble(error)
#     print("L2 error u FEM:",np.sqrt(L2_error))
    
    u_fem_fun = bempp.api.GridFunction(trace_space, fun=f)
    u_fun = bempp.api.GridFunction(trace_space, fun=f)
    lambda_fun = bempp.api.GridFunction(bempp_space, fun=g)

    e_fun = [dirichlet_fun_fem-u_fem_fun,dirichlet_fun-u_fun,neumann_fun-lambda_fun]

#     error = 0
#     # V norm
#     slp = bempp.api.operators.boundary.laplace.single_layer(bempp_space, trace_space, bempp_space)
#     hyp = bempp.api.operators.boundary.laplace.hypersingular(trace_space, bempp_space, trace_space)
#     error += np.sqrt(np.dot(e_fun[2].coefficients.conjugate(),(slp * e_fun[2]).projections(bempp_space)))
#     error += np.sqrt(np.dot(e_fun[1].coefficients.conjugate(),(hyp * e_fun[1]).projections(trace_space)))
#     # D part
#     error += beta**.5 * e_fun[1].l2_norm()

    error_conv.append(max(e_fun[1].l2_norm(), e_fun[0].l2_norm()) + e_fun[2].l2_norm())
    
    FEM_u_error.append(np.sqrt(L2_error))
    BEM_u_error.append(e_fun[1].l2_norm())
    BEM_lambda_error.append(e_fun[2].l2_norm())
    
    error_fem_bem = dirichlet_fun_fem - dirichlet_fun

    error_between.append(error_fem_bem.l2_norm())

Converting from Gmsh format (.msh, .gmsh) to DOLFIN XML format
Expecting 55 vertices
Found all vertices
Expecting 104 cells
Found all cells
Conversion done
0.368606944612


--- Instant: compiling ---


In [ ]:
import matplotlib.pyplot as plt

log_mesh_size = np.log(mesh_size)
inv_mesh_size =  np.true_divide(1, mesh_size)

mult = abs(FEM_u_error[len(mesh_size)//2]/mesh_size[len(mesh_size)//2])
o_mesh_size = np.multiply(mult, mesh_size)

mesh_pow_2 = np.multiply(mesh_size, mesh_size)
mult = 0.7*abs(FEM_u_error[len(mesh_size)//2]/mesh_pow_2[len(mesh_size)//2])
o_mesh_pow_2 = np.multiply(mult, mesh_pow_2)

mesh_pow_3 = np.multiply(mesh_pow_2, mesh_size)
mult = abs(FEM_u_error[len(mesh_size)//2]/mesh_pow_3[len(mesh_size)//2])
o_mesh_pow_3 = np.multiply(mult, mesh_pow_3)

plt.figure( figsize=(10, 10), dpi=100)
# plt.loglog(mesh_size, error_conv, 'ko--', linewidth=3, label='H1 norm error')
plt.loglog(mesh_size, error_between, 'ko-', linewidth=3, label='L2 norm error in between')
plt.loglog(mesh_size, FEM_u_error, 'ko-.', linewidth=3, label='FEM L2 norm error')
plt.loglog(mesh_size, BEM_u_error, 'ks-', linewidth=3, label='BEM L2 norm error')
plt.loglog(mesh_size, BEM_lambda_error, 'ks:', linewidth=3, label='BEM H1 semi-norm error')
# plt.loglog(mesh_size, o_mesh_size, 'k:', linewidth=2, label='O(h)')
plt.loglog(mesh_size, o_mesh_pow_2, 'k--', linewidth=2, label='O(h^2)')
# plt.loglog(mesh_size, o_mesh_pow_3, 'k--', linewidth=2, label='O(h^3)')
plt.legend(loc='lower right')
#plt.title('Sphere - beta = 0, p = 1')
plt.xlabel('log(Mesh size)')
plt.ylabel('log(Error)')
plt.show()

In [ ]:
import matplotlib.pyplot as plt

log_mesh_size = np.log(mesh_size)

plt.figure( figsize=(10, 10), dpi=100)
plt.loglog(mesh_size, iter_num, 'ko--', linewidth=3, label='Our preconditioner')
plt.loglog(mesh_size, iter_num_no_prec, 'ko-', linewidth=3, label='Mass preconditioner')
plt.legend(loc='lower right')
#plt.title('Sphere - beta = 0, p = 1')
plt.xlabel('log(Mesh size)')
plt.ylabel('log(Error)')
plt.show()